## Aggregation
Data aggregation is the process where raw data is gathered and expressed in a summary form for statistical analysis.

For example, raw data can be aggregated over a given time period to provide statistics such as average, minimum, maximum, sum, and count. After the data is aggregated and written to a view or report, you can analyze the aggregated data to gain insights about particular resources or resource groups.

In [2]:
import numpy as np
import pandas as pd

### Built in aggregation methods in Pandas
An aggregation method takes a `Series` of values and returns a single value.

In [12]:
numbers = pd.Series(np.random.randint(low=1, high=100, size=5))
numbers[3] = np.nan
numbers

0    10.0
1     6.0
2    74.0
3     NaN
4    88.0
dtype: float64

In [14]:
# Some of Pandas aggregation methods are:
print(f"{numbers.sum() = }")
print(f"{numbers.count() = }") # number of values
print(f"{numbers.size = }") # number of rows

print(f"{numbers.min() = }")
print(f"{numbers.max() = }")

print(f"{numbers.median() = }")
print(f"{numbers.mean() = }") # ignores na values, thus same result as row below:
print(f"{numbers.sum() / numbers.count() = }")
print(f"{numbers.sum() / numbers.size = }")

print(f"{numbers.mode()[0] = }")

numbers.sum() = 178.0
numbers.count() = 4
numbers.size = 5
numbers.min() = 6.0
numbers.max() = 88.0
numbers.median() = 42.0
numbers.mean() = 44.5
numbers.sum() / numbers.count() = 44.5
numbers.sum() / numbers.size = 35.6
numbers.mode()[0] = 6.0


### When run on a `DataFrame` (multiple `Series`) they return a single value for each `Series`, forming a new `Series`.

In [21]:
numbers_df = pd.DataFrame(np.random.randint(low=1, high=100, size=[5, 5]), columns=["A", "B", "C", "D", "E"])
numbers_df.loc[[0,3], ["B", "E"]] = np.nan
numbers_df

,A,B,C,D,E
0,62,NaN,21,76,NaN
1,20,62.0,15,55,27.0
2,77,35.0,77,93,29.0
3,69,NaN,15,87,NaN
4,55,65.0,15,44,70.0


In [26]:
numbers_df.min()  # default axis is "index" (rows)
numbers_df.min(axis="columns")
numbers_df.min(axis="columns").min() # min value of entire DataFrame
numbers_df.median(axis="index").median()
numbers_df.median(axis="columns").median()

0    21.0
1    15.0
2    29.0
3    15.0
4    15.0
dtype: float64

In [27]:
# count null values in each column
numbers_df.isna().sum(axis="columns")

0    2
1    0
2    0
3    2
4    0
dtype: int64

## Working with real data

In [28]:
autos = pd.read_json("../Data/autos.json")
autos.tail(3)

,aspiration,body-style,bore,city-mpg,compression-ratio,curb-weight,drive-wheels,engine-location,engine-size,engine-type,...,make,normalized-losses,num-of-cylinders,num-of-doors,peak-rpm,price,stroke,symboling,wheel-base,width
202,std,sedan,3.58,18,8.8,3012,rwd,front,173,ohcv,...,volvo,95.0,six,four,5500.0,21485.0,2.87,-1,109.1,68.9
203,turbo,sedan,3.01,26,23.0,3217,rwd,front,145,ohc,...,volvo,95.0,six,four,4800.0,22470.0,3.40,-1,109.1,68.9
204,turbo,sedan,3.78,19,9.5,3062,rwd,front,141,ohc,...,volvo,95.0,four,four,5400.0,22625.0,3.15,-1,109.1,68.9


In [31]:
autos["price"].mean()

13207.129353233831

In [33]:
autos[["length", "width", "height"]].head(3)

,length,width,height
0,168.8,64.1,48.8
1,168.8,64.1,48.8
2,171.2,65.5,52.4


In [57]:
# running mean() on multiple columns (a DataFrame) returns a Series of means.
autos[["length", "width", "height"]].head(3).mean()
autos[["length", "width", "height"]].mean()

volvos = autos[autos["make"] == "volvo"]
volvos[["length", "width", "height"]].mean()

toyotas = autos.query("make == 'toyota'")
toyotas[["length", "width", "height"]].mean()

length    171.934375
width      65.090625
height     53.721875
dtype: float64

### Multiple aggregation

In [54]:
autos[["length", "width", "height"]].agg(["min", "mean", "max"])

numeric_columns = [column for column in autos.columns if autos[column].dtype in ["int64", "float64"]]

autos[numeric_columns].agg(["min", "mean", "max"])

,bore,city-mpg,compression-ratio,curb-weight,engine-size,height,highway-mpg,horsepower,length,normalized-losses,peak-rpm,price,stroke,symboling,wheel-base,width
min,2.540000,13.000000,7.000000,1488.000000,61.000000,47.800000,16.00000,48.000000,141.100000,65.0,4150.000000,5118.000000,2.070000,-2.000000,86.600000,60.300000
mean,3.329751,25.219512,10.142537,2555.565854,126.907317,53.724878,30.75122,104.256158,174.049268,122.0,5125.369458,13207.129353,3.255423,0.834146,98.756585,65.907805
max,3.940000,49.000000,23.000000,4066.000000,326.000000,59.800000,54.00000,288.000000,208.100000,256.0,6600.000000,45400.000000,4.170000,3.000000,120.900000,72.300000


In [ ]:
autos.describe()

### Split-Apply-Combine
- Select a feature to user as key
- Split the dataset into groups for each unique key value
- Apply aggregation to each group
- Combine aggregated data into a new dataset


![image](https://nicholasvadivelu.com/assets/images/posts/groupby/split-apply-combine.svg)

### Group by
Use Pandas `.groupby()` method to select a key and split into groups.

This creates a new `DataFrameGroupBy` object containing the grouped DataFrames.

In [59]:
makes = autos.groupby("make")
makes

In [60]:
type(makes)

pandas.core.groupby.generic.DataFrameGroupBy

In [61]:
# autos["make"].unique()
# autos["make"].nunique()
# autos["make"].value_counts()
len(makes)

22

In [74]:
makes.groups # a dictionary
autos.loc[makes.groups["jaguar"]]
makes.get_group("jaguar") # same result as line above

,aspiration,body-style,bore,city-mpg,compression-ratio,curb-weight,drive-wheels,engine-location,engine-size,engine-type,...,make,normalized-losses,num-of-cylinders,num-of-doors,peak-rpm,price,stroke,symboling,wheel-base,width
47,std,sedan,3.63,15,8.1,4066,rwd,front,258,dohc,...,jaguar,145.0,six,four,4750.0,32250.0,4.17,0,113.0,69.6
48,std,sedan,3.63,15,8.1,4066,rwd,front,258,dohc,...,jaguar,NaN,six,four,4750.0,35550.0,4.17,0,113.0,69.6
49,std,sedan,3.54,13,11.5,3950,rwd,front,326,ohcv,...,jaguar,NaN,twelve,two,5000.0,36000.0,2.76,0,102.0,70.6


### Apply & Combine
It is possible to access a single group as above. However, most of the time, we rather apply aggregation functions to each group individually and combine the result into a new dataset.

In [88]:
makes.count().head(3)
makes[["length", "width", "height"]] # returns a new DataFrameGroupBy
makes[["length", "width", "height"]].mean().head(3) # returns a DataFrame

,length,width,height
make,,,
alfa-romero,169.600000,64.566667,50.000000
audi,183.828571,68.714286,54.428571
bmw,184.500000,66.475000,54.825000


### SeriesGroupBy
Indexing a `DataFrameGroupBy` object with a single column will return a `SeriersGroupBy` object.

In [97]:
sgb = makes["price"]
type(sgb)
sgb.groups
sgb.get_group("jaguar")
sgb.mean().head(3)

autos.groupby("make")["price"].mean().sort_values(ascending=False).head(10)

make
jaguar           34600.000000
mercedes-benz    33647.000000
porsche          31400.500000
bmw              26118.750000
volvo            18063.181818
audi             17859.166667
mercury          16503.000000
alfa-romero      15498.333333
peugot           15489.090909
saab             15223.333333
Name: price, dtype: float64

### Multiple aggregations on `SeriesGroupBy`
Use pandas `.agg()` method on `SeriesGroupBy` to do multiple aggregation on a single feature.

In [106]:
sgb.agg(["min", "mean", "max"])
autos.groupby("make")["price"].agg(["min", "mean", "max"])
autos.groupby("make")["price"].describe().transpose()

make,alfa-romero,audi,bmw,chevrolet,dodge,honda,isuzu,jaguar,mazda,mercedes-benz,...,nissan,peugot,plymouth,porsche,renault,saab,subaru,toyota,volkswagen,volvo
count,3.000000,6.000000,8.000000,3.000000,9.000000,13.000000,2.000000,3.000000,17.000000,8.000000,...,18.000000,11.000000,7.000000,4.000000,2.000000,6.000000,12.000000,32.000000,12.000000,11.000000
mean,15498.333333,17859.166667,26118.750000,6007.000000,7875.444444,8184.692308,8916.500000,34600.000000,10652.882353,33647.000000,...,10415.666667,15489.090909,7963.428571,31400.500000,9595.000000,15223.333333,8541.250000,9885.812500,10077.500000,18063.181818
std,1734.937559,3452.379493,9263.832033,754.421633,2213.386044,2061.672112,3014.396208,2047.559523,3975.682094,6789.560306,...,4477.394200,2246.749673,2395.544257,6528.784343,424.264069,2860.794761,1940.191468,3204.982114,2178.549872,3314.650263
min,13495.000000,13950.000000,16430.000000,5151.000000,5572.000000,5399.000000,6785.000000,32250.000000,5195.000000,25552.000000,...,5499.000000,11900.000000,5572.000000,22018.000000,9295.000000,11850.000000,5118.000000,5348.000000,7775.000000,12940.000000
25%,14997.500000,15800.000000,19958.750000,5723.000000,6377.000000,6855.000000,7850.750000,33900.000000,7395.000000,28230.000000,...,7311.500000,13530.000000,6460.500000,29900.500000,9445.000000,12887.500000,7378.750000,7870.500000,8145.000000,16250.000000
50%,16500.000000,17580.000000,22835.000000,6295.000000,7609.000000,7295.000000,8916.500000,35550.000000,10595.000000,32892.000000,...,8124.000000,16630.000000,7609.000000,33278.000000,9595.000000,15275.000000,7894.000000,9103.000000,9737.500000,18420.000000
75%,16500.000000,18617.500000,32290.000000,6435.000000,8558.000000,9095.000000,9982.250000,35775.000000,11845.000000,36532.000000,...,13499.000000,16987.500000,8439.000000,34778.000000,9745.000000,17490.000000,10019.500000,10973.250000,11768.750000,20265.000000
max,16500.000000,23875.000000,41315.000000,6575.000000,12964.000000,12945.000000,11048.000000,36000.000000,18344.000000,45400.000000,...,19699.000000,18150.000000,12764.000000,37028.000000,9895.000000,18620.000000,11694.000000,17669.000000,13845.000000,22625.000000


### Multiple aggregation on `DataFrameGroupBy`
Using pandas `.agg()` method on a `DataFrameGroupBy` to do multiple aggregations on multiple features will return a multi-index column DataFrame.

In [110]:
makes[["length", "width", "height"]].agg(["min", "mean", "max"])

length                    width                  height  \
                 min        mean    max   min       mean   max    min   
make                                                                    
alfa-romero    168.8  169.600000  171.2  64.1  64.566667  65.5   48.8   
audi           176.6  183.828571  192.7  66.2  68.714286  71.4   52.0   
bmw            176.8  184.500000  197.0  64.8  66.475000  70.9   53.7   
chevrolet      141.1  151.933333  158.8  60.3  62.500000  63.6   52.0   
dodge          157.3  160.988889  174.6  63.8  64.166667  66.3   50.2   
honda          144.6  160.769231  175.4  62.5  64.384615  66.0   50.8   
isuzu          155.9  163.775000  172.6  61.8  63.550000  65.2   51.4   
jaguar         191.7  196.966667  199.6  69.6  69.933333  70.6   47.8   
mazda          159.1  170.805882  177.8  64.2  65.588235  66.5   49.6   
mercedes-benz  180.3  195.262500  208.1  70.3  71.062500  72.0   50.8   
mercury        178.4  178.400000  178.4  68.0  68.000000  68.0   54.8   
mitsubishi     157.3  168.030769  173.2  63.8  65.253846  66.3   49.4   
nissan         162.4  170.988889  184.6  63.8  65.088889  67.9   49.7   
peugot         186.7  191.136364  198.9  68.3  68.390909  68.4   56.0   
plymouth       157.3  164.900000  174.6  63.8  64.271429  66.3   50.2   
porsche        168.9  170.260000  175.7  65.0  67.120000  72.3   50.2   
renault        176.8  179.150000  181.5  66.5  66.550000  66.6   50.5   
saab           186.6  186.600000  186.6  66.5  66.500000  66.5   56.1   
subaru         156.9  168.858333  173.6  63.4  64.950000  65.4   52.5   
toyota         158.7  171.934375  187.8  63.6  65.090625  67.7   52.0   
volkswagen     159.3  172.533333  183.1  64.0  65.616667  66.9   51.4   
volvo          188.8  188.800000  188.8  67.2  67.963636  68.9   55.5   

                                
                    mean   max  
make                            
alfa-romero    50.000000  52.4  
audi           54.428571  55.9  
bmw            54.825000  56.3  
chevrolet      52.400000  53.2  
dodge          51.644444  59.8  
honda          53.238462  58.3  
isuzu          52.225000  53.5  
jaguar         51.133333  52.8  
mazda          53.358824  55.5  
mercedes-benz  55.725000  58.7  
mercury        54.800000  54.8  
mitsubishi     50.692308  51.6  
nissan         53.633333  56.1  
peugot         57.181818  58.7  
plymouth       51.971429  59.8  
porsche        51.100000  51.6  
renault        52.850000  55.2  
saab           56.100000  56.1  
subaru         53.750000  55.7  
toyota         53.721875  59.1  
volkswagen     55.183333  55.7  
volvo          56.236364  57.5

### Custom columns aggregation

In [112]:
autos.groupby("make").agg({"price": "mean", "horsepower": "max"}).head(3)

,price,horsepower
make,,
alfa-romero,15498.333333,154.0
audi,17859.166667,160.0
bmw,26118.750000,182.0


In [146]:
def list_unique(x):
    return ", ".join(sorted(x.apply(str).unique()))

autos.groupby("make").agg(
    average_price = pd.NamedAgg(column="price", aggfunc="mean"),
    min_horsepower = pd.NamedAgg(column="horsepower", aggfunc="min"),
    max_horsepower = pd.NamedAgg(column="horsepower", aggfunc="max"),
    body_styles = pd.NamedAgg(column="body-style", aggfunc=list_unique)
    # body_styles = pd.NamedAgg(column="body-style", aggfunc=lambda x: ", ".join(sorted(x.apply(str).unique())))
).sort_values(by=["average_price"], ascending=False).head(3)

,average_price,min_horsepower,max_horsepower,body_styles
make,,,,
jaguar,34600.0,176.0,262.0,sedan
mercedes-benz,33647.0,123.0,184.0,"convertible, hardtop, sedan, wagon"
porsche,31400.5,143.0,288.0,"convertible, hardtop, hatchback"


### Lambda functions

In [141]:
def my_sorting_func(person):
    return len(person["first-name"])

mylist = [
    {"first-name": "Fredrik", "last-name": "Andersson", "age": 42},
    {"first-name": "Maria", "last-name": "Martinsson", "age": 22},
    {"first-name": "Johan", "last-name": "Karlsson", "age": 27},
    {"first-name": "Anna", "last-name": "Andersson", "age": 39}
]

sorted(mylist, key=lambda person: len(person["first-name"]), reverse=True)

[{'first-name': 'Fredrik', 'last-name': 'Andersson', 'age': 42},
 {'first-name': 'Maria', 'last-name': 'Martinsson', 'age': 22},
 {'first-name': 'Johan', 'last-name': 'Karlsson', 'age': 27},
 {'first-name': 'Anna', 'last-name': 'Andersson', 'age': 39}]

In [144]:
def my_func():
    return "Hello"

my_func = lambda: "Hello"

other_func = my_func

print(other_func())

Hello
